# Ticket Data Analysis

This notebook demonstrates how to analyze the FreshService ticket data using Python's data science tools.

## Loading the Data

First, let's import the necessary libraries and load our ticket data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Set plot styling
plt.style.use('ggplot')
sns.set(style="darkgrid")
%matplotlib inline

In [ ]:
# Define possible data paths to try
possible_paths = [
    # Original path
    'data/fresh_service_tickets',
    # JupyterLite files directory path
    'files/data/fresh_service_tickets',
    # Relative paths
    '../data/fresh_service_tickets',
    './data/fresh_service_tickets',
    # Absolute path within JupyterLite
    '/files/data/fresh_service_tickets'
]

# Load ticket data
tickets = None
for base_path in possible_paths:
    try:
        path1 = f"{base_path}/Tickets_1.csv"
        path2 = f"{base_path}/Tickets_2.csv"
        print(f"Trying to load from: {path1}")
        
        tickets1 = pd.read_csv(path1)
        tickets2 = pd.read_csv(path2)
        
        # Combine the datasets
        tickets = pd.concat([tickets1, tickets2], ignore_index=True)
        print(f"Successfully loaded {len(tickets)} tickets from {base_path}")
        break
    except Exception as e:
        print(f"Could not load from {base_path}: {e}")

if tickets is None:
    print("Failed to load ticket data from any location. Please check file paths.")

## Exploring the Data

Let's take a look at our data structure.

In [ ]:
# Display basic information about the dataset
if tickets is not None:
    print("Dataset shape:", tickets.shape)
    print("\nColumns:")
    for col in tickets.columns:
        print(f"- {col}")

    # Show the first few rows
    tickets.head()
else:
    print("No data available to explore.")

## Data Cleaning and Preparation

Let's clean and prepare our data for analysis.

In [ ]:
# Convert date columns to datetime format
if tickets is not None:
    date_columns = ['created_time', 'resolved_time', 'closed_time', 'due_by_time', 
                    'initial_response_time', 'last_updated_time']

    for col in date_columns:
        if col in tickets.columns:
            try:
                tickets[col] = pd.to_datetime(tickets[col])
            except Exception as e:
                print(f"Could not convert {col}: {e}")

In [ ]:
# Check for missing values
if tickets is not None:
    missing_data = tickets.isnull().sum()
    missing_percent = (missing_data / len(tickets)) * 100

    missing_stats = pd.DataFrame({
        'Missing Values': missing_data,
        'Percentage': missing_percent
    })

    # Display columns with missing values
    missing_stats[missing_stats['Missing Values'] > 0].sort_values('Missing Values', ascending=False)

## Ticket Analysis

Now let's analyze the tickets by different dimensions.

### Tickets by Status

In [ ]:
if tickets is not None and 'status' in tickets.columns:
    # Count tickets by status
    status_counts = tickets['status'].value_counts()
    
    # Create a pie chart
    plt.figure(figsize=(10, 6))
    plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', startangle=90)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.title('Ticket Distribution by Status')
    plt.show()
else:
    print("'status' column not found in the dataset or no data available")

### Tickets by Priority

In [ ]:
if tickets is not None and 'priority' in tickets.columns:
    # Count tickets by priority
    priority_counts = tickets['priority'].value_counts()
    
    # Create a bar chart
    plt.figure(figsize=(10, 6))
    sns.barplot(x=priority_counts.index, y=priority_counts.values)
    plt.title('Ticket Distribution by Priority')
    plt.xlabel('Priority')
    plt.ylabel('Number of Tickets')
    plt.xticks(rotation=45)
    plt.show()
else:
    print("'priority' column not found in the dataset or no data available")

### Tickets by Category

In [ ]:
if tickets is not None and 'category' in tickets.columns:
    # Get the top 10 categories
    category_counts = tickets['category'].value_counts().head(10)
    
    # Create a horizontal bar chart
    plt.figure(figsize=(12, 8))
    sns.barplot(y=category_counts.index, x=category_counts.values)
    plt.title('Top 10 Ticket Categories')
    plt.xlabel('Number of Tickets')
    plt.ylabel('Category')
    plt.show()
else:
    print("'category' column not found in the dataset or no data available")

### Ticket Creation Over Time

In [ ]:
if tickets is not None and 'created_time' in tickets.columns:
    # Extract date from datetime
    tickets['created_date'] = tickets['created_time'].dt.date
    
    # Count tickets by creation date
    daily_tickets = tickets.groupby('created_date').size()
    
    # Create a line chart
    plt.figure(figsize=(15, 6))
    daily_tickets.plot()
    plt.title('Ticket Creation Over Time')
    plt.xlabel('Date')
    plt.ylabel('Number of Tickets')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Monthly ticket volume
    tickets['created_month'] = tickets['created_time'].dt.to_period('M')
    monthly_tickets = tickets.groupby('created_month').size()
    
    plt.figure(figsize=(15, 6))
    monthly_tickets.plot(kind='bar')
    plt.title('Monthly Ticket Volume')
    plt.xlabel('Month')
    plt.ylabel('Number of Tickets')
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.show()
else:
    print("'created_time' column not found in the dataset or no data available")

### Resolution Time Analysis

In [ ]:
if tickets is not None and all(col in tickets.columns for col in ['created_time', 'resolved_time']):
    # Calculate resolution time in hours
    tickets['resolution_time_hours'] = (tickets['resolved_time'] - tickets['created_time']).dt.total_seconds() / 3600
    
    # Filter out negative or extreme values
    valid_resolution = tickets[(tickets['resolution_time_hours'] > 0) & (tickets['resolution_time_hours'] < 1000)]
    
    # Create a histogram
    plt.figure(figsize=(12, 6))
    sns.histplot(valid_resolution['resolution_time_hours'], bins=50, kde=True)
    plt.title('Distribution of Ticket Resolution Time')
    plt.xlabel('Resolution Time (hours)')
    plt.ylabel('Number of Tickets')
    plt.grid(True)
    plt.show()
    
    # Resolution time by priority
    if 'priority' in tickets.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(x='priority', y='resolution_time_hours', data=valid_resolution)
        plt.title('Resolution Time by Priority')
        plt.xlabel('Priority')
        plt.ylabel('Resolution Time (hours)')
        plt.grid(True, axis='y')
        plt.show()
else:
    print("'created_time' or 'resolved_time' columns not found in the dataset or no data available")

## Advanced Analysis

Let's dive deeper into the data to extract more insights.

### SLA Compliance

In [ ]:
if tickets is not None and 'first_response_status' in tickets.columns:
    # Analyze SLA compliance for first response
    sla_counts = tickets['first_response_status'].value_counts()
    
    plt.figure(figsize=(10, 6))
    plt.pie(sla_counts, labels=sla_counts.index, autopct='%1.1f%%', startangle=90)
    plt.axis('equal')
    plt.title('First Response SLA Compliance')
    plt.show()
else:
    print("'first_response_status' column not found in the dataset or no data available")

### Top Requesters

In [ ]:
if tickets is not None and 'requester_name' in tickets.columns:
    # Get top 10 requesters
    top_requesters = tickets['requester_name'].value_counts().head(10)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(y=top_requesters.index, x=top_requesters.values)
    plt.title('Top 10 Ticket Requesters')
    plt.xlabel('Number of Tickets')
    plt.ylabel('Requester')
    plt.grid(True, axis='x')
    plt.show()
else:
    print("'requester_name' column not found in the dataset or no data available")

### Top Agents

In [ ]:
if tickets is not None and 'agents' in tickets.columns:
    # Get top 10 agents
    top_agents = tickets['agents'].value_counts().head(10)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(y=top_agents.index, x=top_agents.values)
    plt.title('Top 10 Ticket Handlers')
    plt.xlabel('Number of Tickets')
    plt.ylabel('Agent')
    plt.grid(True, axis='x')
    plt.show()
else:
    print("'agents' column not found in the dataset or no data available")

## Conclusion

This notebook provides a starting point for analyzing your ticket data. You can extend this analysis by:

1. Adding more visualizations specific to your needs
2. Performing correlation analysis between different metrics
3. Conducting trend analysis over longer periods
4. Implementing machine learning models for ticket classification or resolution time prediction